In [1]:
import xml.dom.minidom as xmldom

In [2]:
path = './zbx_export_templates.xml'

In [3]:
# 解析文件
doc = xmldom.parse(path) 
root = doc.documentElement

In [4]:
# 获得templates下所有template节点
templates = root.getElementsByTagName('template')
node_templates = []
# 遍历节点，选择符合条件，获得当前模板的template名称
for template in templates:
    if (template.parentNode.nodeName == 'templates') and (template.parentNode.parentNode.nodeName == 'zabbix_export'):
        node_templates.append(template)
len(node_templates) # 这里的节点就是templates下每一层template中的内容（即'\n      ')

159

In [7]:
items = []
for template in node_templates:
    
    for index1 in template.childNodes:
    #print(node_templates[67])
        if str(index1).startswith('<DOM Element: discovery_rules'):
            index_rules = template.childNodes.index(index1)
            discovery_rules =  template.childNodes[index_rules]
            #print(discovery_rules)
            for index2 in discovery_rules.childNodes[1::2]:
                dict_item = {}
                nodes = index2.childNodes
                for i in nodes:
                    #print(i)
                    if str(i).startswith('<DOM Element: item_prototypes'):
                        #print(i.childNodes[1].childNodes[1].firstChild.data)
                        if i.childNodes:
                            item_nodes = i.childNodes[1].childNodes
                            for node in item_nodes:
                                if node.nodeName == 'name':
                                    dict_item['itemname'] = node.firstChild.data
                                elif node.nodeName == 'key':
                                    dict_item['key'] = node.firstChild.data
                                elif node.nodeName == 'description':
                                    if node.firstChild:
                                        dict_item['description'] = node.firstChild.data
                                elif node.nodeName == 'value_type':
                                    if node.firstChild:
                                        dict_item['value_type'] = node.firstChild.data
                                elif node.nodeName == 'units':
                                    if node.firstChild:
                                         dict_item['units'] = node.firstChild.data
                                elif node.nodeName == 'delay':
                                    if node.firstChild:
                                         dict_item['delay'] = node.firstChild.data
                                elif node.nodeName == 'applications':
                                    if node.firstChild:
                                        dict_item['application'] = node.childNodes[1].childNodes[1].firstChild.data  
                                    else:
                                        dict_item['application'] = node.parentNode.parentNode.parentNode.childNodes[7].childNodes[1].childNodes[1].firstChild.data
                            dict_item['discovery_rule'] = i.parentNode.childNodes[1].firstChild.data
                            dict_item['template'] = template.childNodes[1].firstChild.data
                            

                items.append(dict_item)
    #print(template.childNodes[1].firstChild.data)

In [8]:
j = []
for i in items:
    if i:
        j.append(i)
len(j)

157

In [10]:
import pandas as pd
discovery = pd.DataFrame(j)
discovery.to_csv('discovery_items.csv', index = False)